In [1]:
import torch
import os
from pprint import pprint
import math
import sys
from pathlib import Path
sys.path.append(os.getcwd())
sys.path.append(str(Path(os.getcwd()).parent))
import training as T

In [2]:
seed = sys.argv[1]
ds_idx = sys.argv[2]

In [8]:
seed = 0
ds_idx = 0

In [9]:
datasets = os.listdir('../ts_datasets/')
datasets = [dataset for dataset in datasets if dataset.endswith('.tsds')]
datasets.sort()
dataset = datasets[ds_idx]
package = torch.load(f'../ts_datasets/{dataset}')

name = package['name']

N_train = package['N_train']
N_valid = package['N_valid']
N_test = package['N_test']

N_class = package['N_class']

N_channel = package['N_channel']
N_length = package['N_length']

N_feature = N_channel * N_length

print(f'dataset: {name}, N_train: {N_train}, N_valid: {N_valid}, N_test: {N_test}, N_class: {N_class}, N_feature: {N_feature}, N_channel: {N_channel}, N_length: {N_length}')

X_train = package['X_train']
X_valid = package['X_valid']
X_test = package['X_test']

y_train = package['Y_train']
y_valid = package['Y_valid']
y_test = package['Y_test']

print(f'X_train: {X_train.shape}, X_valid: {X_valid.shape}, X_test: {X_test.shape}')

dataset: ArrowHead, N_train: 126, N_valid: 42, N_test: 43, N_class: 3, N_feature: 251, N_channel: 1, N_length: 251
X_train: torch.Size([126, 1, 251]), X_valid: torch.Size([42, 1, 251]), X_test: torch.Size([43, 1, 251])


In [10]:
package.keys()

dict_keys(['name', 'X_train', 'Y_train', 'X_valid', 'Y_valid', 'X_test', 'Y_test', 'N_train', 'N_valid', 'N_test', 'N_channel', 'N_length', 'N_class'])

In [11]:
stride = max(2,int(N_class/2))
size = N_class
padding = int((N_class-1)/2)

In [12]:
L_in = N_length
L_outs = []
while L_in > N_class:
    L_out = int((L_in+2*padding-size) / stride) + 1
    L_outs.append(L_out)
    L_in = L_out
L_outs.pop()
N_channels = torch.linspace(N_channel, 1, len(L_outs)+1).round().long()

print(L_outs)

[126, 63, 32, 16, 8, 4]


In [13]:
torch.manual_seed(seed)

model = torch.nn.Sequential()
for i in range(len(L_outs)):
    model.add_module(f'{i}_conv', torch.nn.Conv1d(N_channels[i], N_channels[i+1], N_class, stride=stride, padding=padding))
    model.add_module(f'{i}_relu', torch.nn.PReLU())
model.add_module(f'{i+1}', torch.nn.Flatten())
model.add_module(f'{i+2}', torch.nn.Linear(L_outs[-1], N_class))
model.add_module(f'{i+3}', torch.nn.Softmax(dim=1))
pprint(model)

Sequential(
  (0_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (0_relu): PReLU(num_parameters=1)
  (1_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (1_relu): PReLU(num_parameters=1)
  (2_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (2_relu): PReLU(num_parameters=1)
  (3_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (3_relu): PReLU(num_parameters=1)
  (4_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (4_relu): PReLU(num_parameters=1)
  (5_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (5_relu): PReLU(num_parameters=1)
  (6): Flatten(start_dim=1, end_dim=-1)
  (7): Linear(in_features=4, out_features=3, bias=True)
  (8): Softmax(dim=1)
)


/Users/haibinzhao/miniconda3/envs/ML/lib/python3.8/site-packages/torch/nn/modules/conv.py:132: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  (out_channels, in_channels // groups, *kernel_size), **factory_kwargs))


In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

In [15]:
setup = f'{name}_{seed}.cnn'

In [16]:
best_nn = T.training(model, loss_fn, optimizer, X_train, y_train, X_valid, y_valid, X_test, y_test)

epoch:        0 | train loss: 1.10873e+00 | valid loss: 1.12744e+00 | train acc: 0.3175 | valid acc: 0.2381 | test acc: 0.3488 | patience: 0
epoch:      100 | train loss: 1.10740e+00 | valid loss: 1.12459e+00 | train acc: 0.3175 | valid acc: 0.2381 | test acc: 0.3488 | patience: 0
epoch:      200 | train loss: 1.10619e+00 | valid loss: 1.12186e+00 | train acc: 0.3175 | valid acc: 0.2381 | test acc: 0.3488 | patience: 0
epoch:      300 | train loss: 1.10496e+00 | valid loss: 1.11900e+00 | train acc: 0.3175 | valid acc: 0.2381 | test acc: 0.3488 | patience: 0
epoch:      400 | train loss: 1.10373e+00 | valid loss: 1.11604e+00 | train acc: 0.3175 | valid acc: 0.2381 | test acc: 0.3488 | patience: 0
epoch:      500 | train loss: 1.10253e+00 | valid loss: 1.11304e+00 | train acc: 0.3175 | valid acc: 0.2381 | test acc: 0.3488 | patience: 0
epoch:      600 | train loss: 1.10140e+00 | valid loss: 1.11005e+00 | train acc: 0.3175 | valid acc: 0.2381 | test acc: 0.3488 | patience: 0
epoch:      7

KeyboardInterrupt: 

In [194]:
acc_train = (best_nn(X_train).argmax(dim=1) == y_train).float().mean()
acc_valid = (best_nn(X_valid).argmax(dim=1) == y_valid).float().mean()
acc_test = (best_nn(X_test).argmax(dim=1) == y_test).float().mean()

package = {'name': name, model: best_nn, 'acc_train': acc_train, 'acc_valid': acc_valid, 'acc_test': acc_test}
print(package)

if not os.path.exists('./baseline_result/CNN/'):
    os.makedirs('./baseline_result/CNN/')
torch.save(package, f'./baseline_result/CNN/{setup}')


{'name': 'AtrialFibrillation', Sequential(
  (0_conv): Conv1d(2, 2, kernel_size=(3,), stride=(2,), padding=(1,))
  (0_relu): Tanh()
  (1_conv): Conv1d(2, 2, kernel_size=(3,), stride=(2,), padding=(1,))
  (1_relu): Tanh()
  (2_conv): Conv1d(2, 2, kernel_size=(3,), stride=(2,), padding=(1,))
  (2_relu): Tanh()
  (3_conv): Conv1d(2, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (3_relu): Tanh()
  (4_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (4_relu): Tanh()
  (5_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (5_relu): Tanh()
  (6_conv): Conv1d(1, 1, kernel_size=(3,), stride=(2,), padding=(1,))
  (6_relu): Tanh()
  (7): Flatten(start_dim=1, end_dim=-1)
  (8): Linear(in_features=5, out_features=3, bias=True)
  (9): Softmax(dim=1)
): Sequential(
  (0_conv): Conv1d(2, 2, kernel_size=(3,), stride=(2,), padding=(1,))
  (0_relu): Tanh()
  (1_conv): Conv1d(2, 2, kernel_size=(3,), stride=(2,), padding=(1,))
  (1_relu): Tanh()
  (2_conv): Conv1d(2, 2